In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time

from datetime import date, datetime
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

sns.set()

In [25]:
df = pd.read_excel('/content/Job_Data_CareerViet.xlsx')

In [26]:
df.head(10)

,Unnamed: 0,Job_Name,Company_name,Location,Salary,Time,Benefit,Description,Requirement,URL
0,0,Chuyên Viên Hỗ Trợ Kỹ Thuật ( ô tô thương mại)...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,2025-10-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",• Hỗ trợ kỹ thuật & hướng dẫn sửa chữa cho hệ ...,• Tốt nghiệp đại học ngành Cơ khí ô tô. • Tối ...,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...
1,1,CHUYÊN VIÊN PHỤ TÙNG (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,8 Tr - 20 Tr VND,2025-09-30 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","- Xây dựng chính sách, kế hoạch và KPI kinh do...","-Tốt nghiệp ĐH ngành Cơ khí ô tô, Logistics, K...",https://careerviet.vn/vi/tim-viec-lam/chuyen-v...
2,2,PHÓ TRƯỞNG BỘ PHẬN PHÁP LÝ – ĐĂNG KIỂM XE CBU/...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,2025-08-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",Mô tả công việc: - Quản lý hồ sơ pháp lý – kỹ ...,"- Tốt nghiệp kỹ thuật ô tô, cơ khí, luật hoặc ...",https://careerviet.vn/vi/tim-viec-lam/pho-truo...
3,3,Tài Xế Lái Xe Cho Giám Đốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,8 Tr - 10 Tr VND,2025-06-30 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","Mô tả công việc Phụ trách đưa đón, chuyên chở ...","Nam, sức khỏe tốt, không bị cận; GPLX hạng B2 ...",https://careerviet.vn/vi/tim-viec-lam/tai-xe-l...
4,4,Phiên dịch viên tiếng Trung Quốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,18 Tr - 25 Tr VND,2025-08-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...","- Phiên dịch, biên dịch tài liệu kỹ thuật và n...",- Thành thạo tiếng Trung - Ưu tiên ứng viên có...,https://careerviet.vn/vi/tim-viec-lam/phien-di...
5,5,Trưởng Nhóm Nhân Sự (MỚI),CT CP TC UAV,Bình Dương,20 Tr - 25 Tr VND,2025-06-07 00:00:00,"Laptop, Chế độ bảo hiểm, Du Lịch, Chế độ thưởn...",NHIỆM VỤ CHÍNH 1. Chiến lược nhân sự Tham mưu ...,YÊU CẦU CÔNG VIỆC Tốt nghiệp đại học trở lên c...,https://careerviet.vn/vi/tim-viec-lam/truong-n...
6,6,Giám sát chuỗi cửa hàng - Ngành thời trang (MỚI),Công Ty TNHH Thời Trang Gior (Nhãn Hàng Thời T...,Hà Nội,Cạnh tranh,2025-07-07 00:00:00,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...",- Đảm bảo tình hình nhân sự ổn định tại các cử...,Yêu cầu công việc: - Kinh nghiệm 2 năm ở vị tr...,https://careerviet.vn/vi/tim-viec-lam/giam-sat...
7,7,Nhân viên Vận hành Livestream | Livestream Ope...,N-SQUARED ECOMMERCE VIETNAM,Hồ Chí Minh,Cạnh tranh,2025-07-06 00:00:00,"Laptop, Chế độ bảo hiểm, Chăm sóc sức khỏe, Đà...",1. Vận hành Livestream / Livestream Operation ...,"- Có kinh nghiệm với livestream, quản lý KOC/A...",https://careerviet.vn/vi/tim-viec-lam/nhan-vie...
8,8,KỸ THUẬT VIÊN X - QUANG (MỚI),SCI Group,Hà Nội,10 Tr - 14 Tr VND,2025-07-06 00:00:00,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Đồng phục, ...",- Thực hiện các kỹ thuật chuyên X Quang theo c...,- Ứng viên Nam/Nữ/LGBT; Tốt nghiệp đúng chuyên...,https://careerviet.vn/vi/tim-viec-lam/ky-thuat...
9,9,"Nhân Viên Hành Chính, Cơ Sở Vật Chất (MỚI)",StudyLink Company,Hồ Chí Minh,"6,5 Tr - 7,5 Tr VND",2025-07-07 00:00:00,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Chế độ thưở...","- Phụ trách mua sắm, thuê các tài sản, t...","Yêu cầu: - Sinh viên mới ra trường, chưa nhiều...",https://careerviet.vn/vi/tim-viec-lam/nhan-vie...


In [27]:
df = df.drop('Unnamed: 0', axis=1)
display(df.head())

,Job_Name,Company_name,Location,Salary,Time,Benefit,Description,Requirement,URL
0,Chuyên Viên Hỗ Trợ Kỹ Thuật ( ô tô thương mại)...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,2025-10-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",• Hỗ trợ kỹ thuật & hướng dẫn sửa chữa cho hệ ...,• Tốt nghiệp đại học ngành Cơ khí ô tô. • Tối ...,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...
1,CHUYÊN VIÊN PHỤ TÙNG (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,8 Tr - 20 Tr VND,2025-09-30 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","- Xây dựng chính sách, kế hoạch và KPI kinh do...","-Tốt nghiệp ĐH ngành Cơ khí ô tô, Logistics, K...",https://careerviet.vn/vi/tim-viec-lam/chuyen-v...
2,PHÓ TRƯỞNG BỘ PHẬN PHÁP LÝ – ĐĂNG KIỂM XE CBU/...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,2025-08-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",Mô tả công việc: - Quản lý hồ sơ pháp lý – kỹ ...,"- Tốt nghiệp kỹ thuật ô tô, cơ khí, luật hoặc ...",https://careerviet.vn/vi/tim-viec-lam/pho-truo...
3,Tài Xế Lái Xe Cho Giám Đốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,8 Tr - 10 Tr VND,2025-06-30 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","Mô tả công việc Phụ trách đưa đón, chuyên chở ...","Nam, sức khỏe tốt, không bị cận; GPLX hạng B2 ...",https://careerviet.vn/vi/tim-viec-lam/tai-xe-l...
4,Phiên dịch viên tiếng Trung Quốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,18 Tr - 25 Tr VND,2025-08-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...","- Phiên dịch, biên dịch tài liệu kỹ thuật và n...",- Thành thạo tiếng Trung - Ưu tiên ứng viên có...,https://careerviet.vn/vi/tim-viec-lam/phien-di...


In [28]:
import re
from tqdm import tqdm

def normalize_text_job(text):
  text = str(text) # Convert to string to handle non-string types like NaN
  text = re.sub('\n+', ' ', text)
  text = re.sub('\xa0', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('\t', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  text = text.strip()
  return text

def normalize_salary_job(salary):
  salary = str(salary) # Convert to string to handle non-string types like NaN
  salary = salary.replace('.', '')
  salary = salary.replace(',', '')
  salary = salary.replace('Thoả thuận', 'Thỏa thuận')
  result = []
  if bool(re.search(r'\d', salary)):
    if 'Tr' in salary:
      result.extend([int(j)*1000000 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    elif 'USD' in salary:
      result.extend([int(j)*23720 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    else:
      result.extend([int(j)*1000000 if int(j) < 100000 else int(j) for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
  else:
    result.append(salary)

  return result

def normalize_experience_job(experience):
  try:
    result = 'Không yêu cầu kinh nghiệm'
    experience = str(experience) # Convert to string to handle non-string types like NaN
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', experience)[0])
    if bool(re.search(r'\d', experience)):
      if 'Dưới' in experience or number < 1:
        result = 'Dưới 1 năm'
      elif number < 3:
        result = '1-3 năm'
      elif number < 5:
        result = '3-5 năm'
      elif number < 10:
        result = '5-10 năm'
      else:
        result = 'Trên 10 năm'
  except:
    result = 'Không yêu cầu kinh nghiệm'

  return result


def normalize_date(date_str):
    """
    Chuẩn hóa một chuỗi ngày tháng từ các định dạng khác nhau
    về định dạng chuẩn 'DD-MM-YYYY'.

    Args:
        date_str (str): Chuỗi ngày tháng cần chuẩn hóa (ví dụ: '1/6/2025' hoặc '31-05-2025').

    Returns:
        str: Chuỗi ngày tháng đã được chuẩn hóa ('DD-MM-YYYY') hoặc chuỗi gốc nếu không thể chuyển đổi.
    """
    # Chuyển đổi đầu vào thành chuỗi để xử lý đồng nhất và loại bỏ khoảng trắng thừa
    date_str = str(date_str).strip()

    # Danh sách các định dạng ngày tháng mà chúng ta muốn thử
    # %d: ngày, %m: tháng, %Y: năm (4 chữ số)
    formats_to_try = [
        '%d-%m-%Y',  # Ví dụ: 31-05-2025
        '%d/%m/%Y'   # Ví dụ: 1/6/2025
    ]

    for fmt in formats_to_try:
        try:
            # Thử chuyển đổi chuỗi thành đối tượng datetime theo từng định dạng
            parsed_date = datetime.strptime(date_str, fmt)
            # Nếu thành công, định dạng lại thành chuỗi 'DD-MM-YYYY' và trả về
            return parsed_date.strftime('%d-%m-%Y')
        except ValueError:
            # Nếu định dạng không khớp, bỏ qua và thử định dạng tiếp theo
            continue

    # Nếu không có định dạng nào khớp, trả về chuỗi gốc để kiểm tra thủ công
    return date_str


def processing_job_data(data):
  # Xử lý dữ liệu trên từng cột
  for i in tqdm(range(data.shape[0])):

      # Xử lý cột Description
      data.loc[i,'Description'] = re.sub('Description', '', normalize_text_job(data.loc[i,'Description']))
      # Xử lý cột Requirement
      data.loc[i,'Requirement'] = re.sub('Requirement', '', normalize_text_job(data.loc[i,'Requirement']))
      # Xử lý cột Benefit
      data.loc[i,'Benefit'] = re.sub('Benefit', '', normalize_text_job(data.loc[i,'Benefit']))
      # Xử lý cột Time
      data.loc[i, 'Time'] = normalize_date(data.loc[i, 'Time'])

      # Xử lý cột Salary
      salary = min(normalize_salary_job(data.loc[i,'Salary']))
      try:
        if salary < 5000000:
          data.loc[i,'Salary'] = '1,000,000 - 5,000,000'
        elif salary < 10000000:
          data.loc[i,'Salary'] = '5,000,000 - 10,000,000'
        elif salary < 15000000:
          data.loc[i,'Salary'] = '10,000,000 - 15,000,000'
        elif salary < 20000000:
          data.loc[i,'Salary'] = '15,000,000 - 20,000,000'
        elif salary < 40000000:
          data.loc[i,'Salary'] = '20,000,000 - 40,000,000'
        else:
          data.loc[i,'Salary'] = 'Trên 40,000,000'
      except:
        data.loc[i,'Salary'] = salary


  return data

In [29]:
%%time
processing_job_data(df)

100%|██████████| 21861/21861 [00:17<00:00, 1275.00it/s]


CPU times: user 16.6 s, sys: 170 ms, total: 16.8 s
Wall time: 17.2 s


,Job_Name,Company_name,Location,Salary,Time,Benefit,Description,Requirement,URL
0,Chuyên Viên Hỗ Trợ Kỹ Thuật ( ô tô thương mại)...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,2025-10-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",• Hỗ trợ kỹ thuật & hướng dẫn sửa chữa cho hệ ...,• Tốt nghiệp đại học ngành Cơ khí ô tô. • Tối ...,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...
1,CHUYÊN VIÊN PHỤ TÙNG (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,"5,000,000 - 10,000,000",2025-09-30 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","- Xây dựng chính sách, kế hoạch và KPI kinh do...","-Tốt nghiệp ĐH ngành Cơ khí ô tô, Logistics, K...",https://careerviet.vn/vi/tim-viec-lam/chuyen-v...
2,PHÓ TRƯỞNG BỘ PHẬN PHÁP LÝ – ĐĂNG KIỂM XE CBU/...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,2025-08-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",Mô tả công việc: - Quản lý hồ sơ pháp lý – kỹ ...,"- Tốt nghiệp kỹ thuật ô tô, cơ khí, luật hoặc ...",https://careerviet.vn/vi/tim-viec-lam/pho-truo...
3,Tài Xế Lái Xe Cho Giám Đốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,"5,000,000 - 10,000,000",2025-06-30 00:00:00,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","Mô tả công việc Phụ trách đưa đón, chuyên chở ...","Nam, sức khỏe tốt, không bị cận; GPLX hạng B2 ...",https://careerviet.vn/vi/tim-viec-lam/tai-xe-l...
4,Phiên dịch viên tiếng Trung Quốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,"15,000,000 - 20,000,000",2025-08-31 00:00:00,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...","- Phiên dịch, biên dịch tài liệu kỹ thuật và n...",- Thành thạo tiếng Trung - Ưu tiên ứng viên có...,https://careerviet.vn/vi/tim-viec-lam/phien-di...
...,...,...,...,...,...,...,...,...,...
21856,SENIOR ORGANIZATIONAL DEVELOPMENT EXECUTIVE_HCM,CÔNG TY TNHH TRANSCOSMOS VIỆT NAM,Hồ Chí Minh,Cạnh tranh,2025-05-17 00:00:00,Chế độ bảo hiểm Du Lịch Phụ cấp Chế độ thưởng ...,"WE NEED YOU:Analyzing business structures, pro...",Bachelor's at relevant experience.A minimum of...,https://careerviet.vn/vi/tim-viec-lam/senior-o...
21857,[Toàn Quốc] Giám Sát Kinh Doanh Trực Tiếp,Công Ty Tài Chính Mirae Asset (Việt Nam),Đồng Bằng Sông Cửu Long,Cạnh tranh,2025-05-17 00:00:00,Laptop Chế độ bảo hiểm Du Lịch Phụ cấp Du lịch...,Tuyển dụng và đào tạo nhân viên kinh doanh về ...,"Tốt nghiệp Trung cấp trở lên, đã từng làm việc...",https://careerviet.vn/vi/tim-viec-lam/toan-quo...
21858,HO - TBP/Chuyên gia Phòng chống rửa tiền và tu...,Ngân Hàng TMCP Phương Đông - OCB,Hồ Chí Minh,Cạnh tranh,2025-05-17 00:00:00,Chế độ bảo hiểm Du Lịch Phụ cấp Đồng phục Chế ...,- Tham gia xây dựng chính sách quản lý việc tu...,- Tốt nghiệp Đại học trở lên chuyên ngành Ngân...,https://careerviet.vn/vi/tim-viec-lam/ho-tbp-c...
21859,Chuyên viên cao cấp Nội dung truyền thông,Ngân Hàng TMCP Sài Gòn - Hà Nội ( SHB ),Hà Nội,Cạnh tranh,2025-05-17 00:00:00,nan,"Lập kế hoạch tuyến bài, sản xuất tin bài về sả...","Tốt nghiệp Đại học chuyên ngành báo chí, truyề...",https://careerviet.vn/vi/tim-viec-lam/chuyen-v...


In [30]:
df.replace('nan', np.nan, inplace=True)

# Sau khi chạy lệnh trên, bạn hãy kiểm tra lại
print("Số lượng giá trị rỗng sau khi sửa lỗi:")
print(df.isna().sum())

Số lượng giá trị rỗng sau khi sửa lỗi:
Job_Name          0
Company_name      0
Location          0
Salary            0
Time            478
Benefit         612
Description     308
Requirement     265
URL               0
dtype: int64


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21861 entries, 0 to 21860
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job_Name      21861 non-null  object
 1   Company_name  21861 non-null  object
 2   Location      21861 non-null  object
 3   Salary        21861 non-null  object
 4   Time          21383 non-null  object
 5   Benefit       21249 non-null  object
 6   Description   21553 non-null  object
 7   Requirement   21596 non-null  object
 8   URL           21861 non-null  object
dtypes: object(9)
memory usage: 1.5+ MB


In [32]:
csv_output_file = '/content/job_data_Careerviet_processed.csv'

In [33]:
df.to_csv(csv_output_file, index=False, encoding='utf-8-sig')